# 긴 문서 콘텐츠 추출

GPT-3는 너무 커서 컨텍스트 창에 맞지 않는 문서에서 주요 수치, 날짜 또는 기타 중요한 콘텐츠를 추출하는 데 도움이 될 수 있습니다. 이 문제를 해결하기 위한 한 가지 접근 방식은 문서를 청크로 분할하고 각 청크를 개별적으로 처리한 다음 하나의 답변 목록으로 결합하는 것입니다. 

이 노트북에서는 이 방법을 실행해 보겠습니다:
- 긴 PDF를 불러와서 텍스트 추출하기
- 핵심 정보를 추출하는 데 사용할 프롬프트 만들기
- 문서를 청크 단위로 나누고 각 청크를 처리해 답을 추출합니다.
- 마지막에 이들을 결합합니다.
- 이 간단한 접근 방식을 세 가지 더 어려운 질문으로 확장합니다.

GPT-3 can help us extract key figures, dates or other bits of important content from documents that are too big to fit into the context window. One approach for solving this is to chunk the document up and process each chunk separately, before combining into one list of answers. 

In this notebook we'll run through this approach:
- Load in a long PDF and pull the text out
- Create a prompt to be used to extract key bits of information
- Chunk up our document and process each chunk to pull any answers out
- Combine them at the end
- This simple approach will then be extended to three more difficult questions

## 접근 방식

- **설정**: 전력 단위에 관한 금융 규정 문서인 PDF를 가져와서 엔티티 추출을 위해 텍스트를 추출합니다. 이를 사용하여 콘텐츠에 묻혀 있는 답을 추출해 보겠습니다.
- **간단한 엔티티 추출**: 문서 덩어리에서 핵심 정보 비트를 추출합니다:
    - 질문과 예상되는 형식의 예가 포함된 템플릿 프롬프트 만들기
    - 텍스트 청크를 입력으로 받아 프롬프트와 결합하여 응답을 가져오는 함수를 만듭니다.
    - 스크립트를 실행하여 텍스트를 청크하고 답변을 추출하여 구문 분석을 위해 출력하기
- **복잡한 엔티티 추출**: 더 어려운 추론이 필요한 몇 가지 더 어려운 질문을 해보세요.

- **Setup**: Take a PDF, a Formula 1 Financial Regulation document on Power Units, and extract the text from it for entity extraction. We'll use this to try to extract answers that are buried in the content.
- **Simple Entity Extraction**: Extract key bits of information from chunks of a document by:
    - Creating a template prompt with our questions and an example of the format it expects
    - Create a function to take a chunk of text as input, combine with the prompt and get a response
    - Run a script to chunk the text, extract answers and output them for parsing
- **Complex Entity Extraction**: Ask some more difficult questions which require tougher reasoning to work out

## Setup

In [22]:
!pip install textract
!pip install tiktoken

In [15]:
import textract
import os
import openai
import tiktoken

# Extract the raw text from each PDF using textract
# text = textract.process('data/websquare_snippet_guide.pdf', method='pdfminer').decode('utf-8')
text = textract.process('data/fia_f1_power_unit_financial_regulations_issue_1_-_2022-08-16.pdf', method='pdfminer').decode('utf-8')
clean_text = text.replace("  ", " ").replace("\n", "; ").replace(';',' ')
# print(text)

## 간단한 엔티티 추출

In [16]:
# Example prompt - 
document = '<document>'
template_prompt=f'''Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output \"Not specified\".
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Who is the author\n1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR\n2. What is the value of External Manufacturing Costs in USD\n3. What is the Capital Expenditure Limit in USD\n\nDocument: \"\"\"{document}\"\"\"\n\n0. Who is the author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output "Not specified".
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Who is the author
1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR
2. What is the value of External Manufacturing Costs in USD
3. What is the Capital Expenditure Limit in USD

Document: """<document>"""

0. Who is the author: Tom Anderson (Page 1)
1.


In [17]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """Yield successive n-sized chunks from text."""
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

def extract_chunk(document,template_prompt):
    
    prompt=template_prompt.replace('<document>',document)

    response = openai.Completion.create(
    model='text-davinci-003', 
    prompt=prompt,
    temperature=0,
    max_tokens=1500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return "1." + response['choices'][0]['text']

In [18]:
# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

results = []
    
chunks = create_chunks(clean_text,1000,tokenizer)
text_chunks = [tokenizer.decode(chunk) for chunk in chunks]

for chunk in text_chunks:
    results.append(extract_chunk(chunk,template_prompt))
    #print(chunk)
    print(results[-1])


1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: Not specified
2. What is the value of External Manufacturing Costs in USD: Not specified
3. What is the Capital Expenditure Limit in USD: Not specified
1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: USD 95,000,000 (Page 2); GBP 76,459,000 (Page 2); EUR 90,210,000 (Page 2)
2. What is the value of External Manufacturing Costs in USD: Not specified
3. What is the Capital Expenditure Limit in USD: Not specified
1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: Not specified
2. What is the value of External Manufacturing Costs in USD: Not specified
3. What is the Capital Expenditure Limit in USD: Not specified
1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: Not specified
2. What is the value of External Manufacturing Costs in USD: US Dollars 20,000,000 in respect of each of the Full Year Reporting Periods ending on 31 December 2023, 31 December 2024 

In [19]:
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: USD 95,000,000 (Page 2); GBP 76,459,000 (Page 2); EUR 90,210,000 (Page 2)',
 '2. What is the value of External Manufacturing Costs in USD: US Dollars 20,000,000 in respect of each of the Full Year Reporting Periods ending on 31 December 2023, 31 December 2024 and 31 December 2025, adjusted for Indexation (Page 10)',
 '3. What is the Capital Expenditure Limit in USD: US Dollars 30,000,000 (Page 32)']

## 복합 엔티티 추출

In [20]:
# Example prompt - 
template_prompt=f'''Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output \"Not specified\".
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Who is the author\n1. How is a Minor Overspend Breach calculated\n2. How is a Major Overspend Breach calculated\n3. Which years do these financial regulations apply to\n\nDocument: \"\"\"{document}\"\"\"\n\n0. Who is the author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output "Not specified".
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Who is the author
1. How is a Minor Overspend Breach calculated
2. How is a Major Overspend Breach calculated
3. Which years do these financial regulations apply to

Document: """<document>"""

0. Who is the author: Tom Anderson (Page 1)
1.


In [21]:
results = []

for chunk in text_chunks:
    results.append(extract_chunk(chunk,template_prompt))
    
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. How is a Minor Overspend Breach calculated: A Minor Overspend Breach arises when a Power Unit Manufacturer submits its Full Year Reporting Documentation and Relevant Costs reported therein exceed the Power Unit Cost Cap by less than 5% (Page 24)',
 '2. How is a Major Overspend Breach calculated: A Material Overspend Breach arises when a Power Unit Manufacturer submits its Full Year Reporting Documentation and Relevant Costs reported therein exceed the Power Unit Cost Cap by 5% or more (Page 25)',
 '3. Which years do these financial regulations apply to: 2026 onwards (Page 1)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 and subsequent Full Year Reporting Periods (Page 2)',
 '3. Which years do these financial regulations apply to: 2022-2025 (Page 6)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 and subsequent Full Year Reporting Periods (Page 10)',
 '3. Which years do these financial regulations apply to: 202

## 통합

처음 두 개의 답은 무사히 추출할 수 있었지만 세 번째 답은 모든 페이지에 표시된 날짜 때문에 혼란스러웠지만 정답도 그 안에 있습니다.

이 문제를 더 개선하려면 다음과 같은 실험을 고려해 볼 수 있습니다:
- 보다 설명적이거나 구체적인 프롬프트
- 충분한 학습 데이터가 있는 경우 모델을 미세 조정하여 일련의 출력을 잘 찾도록 합니다.
- 데이터를 청크하는 방식 - 중복되지 않는 1000개의 토큰을 사용했지만, 정보를 섹션으로 나누거나 토큰별로 잘라내는 등 보다 지능적으로 청크하면 더 나은 결과를 얻을 수 있습니다.

그러나 최소한의 조정으로 이제 긴 문서의 내용을 사용하여 다양한 난이도의 6가지 질문에 대한 답을 얻었으며, 엔티티 추출이 필요한 모든 긴 문서에 적용할 수 있는 재사용 가능한 접근 방식을 갖게 되었습니다. 여러분이 이 방법으로 무엇을 할 수 있을지 기대해 주세요!

We've been able to extract the first two answers safely, while the third was confounded by the date that appeared on every page, though the correct answer is in there as well.

To tune this further you can consider experimenting with:
- A more descriptive or specific prompt
- If you have sufficient training data, fine-tuning a model to find a set of outputs very well
- The way you chunk your data - we have gone for 1000 tokens with no overlap, but more intelligent chunking that breaks info into sections, cuts by tokens or similar may get better results

However, with minimal tuning we have now answered 6 questions of varying difficulty using the contents of a long document, and have a reusable approach that we can apply to any long document requiring entity extraction. Look forward to seeing what you can do with this!